In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from keras import regularizers
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.applications.mobilenet_v2 import preprocess_input
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import cv2

Using plaidml.keras.backend backend.


In [2]:
def get_pickles():
    combined_dict = pickle.load(open('../Pickles/TTSDict.p', 'rb'))
    
    x_train = combined_dict['train']['images']
    x_test = combined_dict['test']['images']
    
    y_train = combined_dict['train']['labels']
    y_test = combined_dict['test']['labels']
    
    validation = combined_dict['validation']
    
    return x_train, x_test, y_train, y_test, validation 

x_train, x_test, y_train, y_test, validation  = get_pickles() 

In [3]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(10000, 250, 250, 3) (992, 250, 250, 3)
(10000, 2) (992, 2)


In [4]:
def get_conv_model(dim):
    '''This function will create and compile a CNN given the input dimension'''
    inp_shape = dim
    act = 'relu'
    drop = .25
    kernal_reg = regularizers.l1(.001)
    optimizer = Adam(lr = .0001)
    
    model = Sequential() 
    
    model.add(Conv2D(64, kernel_size=(3,3),activation=act, input_shape = inp_shape, 
                     kernel_regularizer = kernal_reg,
                     kernel_initializer = 'he_uniform',  padding = 'same', name = 'Input_Layer'))
    model.add(MaxPooling2D(pool_size=(2, 2),  strides = (3,3)))
    
    
    model.add(Conv2D(64, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (3,3)))
    

    
    model.add(Conv2D(128, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(Conv2D(128, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (3,3)))

    model.add(Flatten())

    
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))

    model.add(Dropout(drop))

    model.add(Dense(2, activation='sigmoid', name = 'Output_Layer'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=10, min_delta = .00075)
model_checkpoint = ModelCheckpoint('ModelWeights/Normal1.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 5, mode = 'min')
epochs = 2000
batch_size = 100
    

dim = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
normal_model = get_conv_model(dim =dim)

    
augmentation =ImageDataGenerator(rotation_range = 20, width_shift_range = .2, height_shift_range = .2, 
                                                       horizontal_flip = True, shear_range = .15, 
                                 fill_mode = 'nearest', zoom_range = .15)
augmentation.fit(x_train)
normal_history = normal_model.fit_generator(augmentation.flow(x_train, y_train, batch_size = batch_size),
            epochs = epochs, 
     callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"


Epoch 1/2000


INFO:plaidml:Analyzing Ops: 91 of 395 operations complete


100/100 [==============================] - 118s 1s/step - loss: 10.9131 - acc: 0.7653 - val_loss: 9.7655 - val_acc: 0.9173

Epoch 00001: val_loss improved from inf to 9.76555, saving model to ModelWeights/Normal1.h5
Epoch 2/2000
100/100 [==============================] - 104s 1s/step - loss: 9.0291 - acc: 0.9031 - val_loss: 8.2133 - val_acc: 0.9360

Epoch 00002: val_loss improved from 9.76555 to 8.21333, saving model to ModelWeights/Normal1.h5
Epoch 3/2000
100/100 [==============================] - 110s 1s/step - loss: 7.5780 - acc: 0.9255 - val_loss: 6.8846 - val_acc: 0.9415

Epoch 00003: val_loss improved from 8.21333 to 6.88458, saving model to ModelWeights/Normal1.h5
Epoch 4/2000
100/100 [==============================] - 113s 1s/step - loss: 6.3501 - acc: 0.9317 - val_loss: 5.7484 - val_acc: 0.9521

Epoch 00004: val_loss improved from 6.88458 to 5.74837, saving model to ModelWeights/Normal1.h5
Epoch 5/2000
100/100 [==============================] - 110s 1s/step - loss: 5.3118 - ac

100/100 [==============================] - 112s 1s/step - loss: 0.8986 - acc: 0.9624 - val_loss: 0.8654 - val_acc: 0.9698

Epoch 00036: val_loss improved from 0.87473 to 0.86540, saving model to ModelWeights/Normal1.h5
Epoch 37/2000
100/100 [==============================] - 111s 1s/step - loss: 0.8783 - acc: 0.9635 - val_loss: 0.8371 - val_acc: 0.9682

Epoch 00037: val_loss improved from 0.86540 to 0.83705, saving model to ModelWeights/Normal1.h5
Epoch 38/2000
100/100 [==============================] - 110s 1s/step - loss: 0.8528 - acc: 0.9656 - val_loss: 0.8149 - val_acc: 0.9713

Epoch 00038: val_loss improved from 0.83705 to 0.81488, saving model to ModelWeights/Normal1.h5
Epoch 39/2000
100/100 [==============================] - 112s 1s/step - loss: 0.8312 - acc: 0.9643 - val_loss: 0.8059 - val_acc: 0.9698

Epoch 00039: val_loss improved from 0.81488 to 0.80588, saving model to ModelWeights/Normal1.h5
Epoch 40/2000
100/100 [==============================] - 111s 1s/step - loss: 0.80


Epoch 00072: val_loss improved from 0.46265 to 0.45952, saving model to ModelWeights/Normal1.h5
Epoch 73/2000
100/100 [==============================] - 112s 1s/step - loss: 0.4900 - acc: 0.9665 - val_loss: 0.4658 - val_acc: 0.9703

Epoch 00073: val_loss did not improve from 0.45952
Epoch 74/2000
100/100 [==============================] - 112s 1s/step - loss: 0.4768 - acc: 0.9680 - val_loss: 0.4556 - val_acc: 0.9763

Epoch 00074: val_loss improved from 0.45952 to 0.45555, saving model to ModelWeights/Normal1.h5
Epoch 75/2000
100/100 [==============================] - 111s 1s/step - loss: 0.4750 - acc: 0.9682 - val_loss: 0.4499 - val_acc: 0.9763

Epoch 00075: val_loss improved from 0.45555 to 0.44986, saving model to ModelWeights/Normal1.h5
Epoch 76/2000
100/100 [==============================] - 113s 1s/step - loss: 0.4634 - acc: 0.9702 - val_loss: 0.4508 - val_acc: 0.9698

Epoch 00076: val_loss did not improve from 0.44986
Epoch 77/2000
100/100 [==============================] - 112s

In [ ]:
assert False

### Mobilenet

In [ ]:
def get_mobilenet(dim):
    model = Sequential()
    optimizer = Adam(lr = .0005)
    baseModel = MobileNetV2(weights="imagenet", include_top=False,
        input_tensor=Input(shape=dim))
    
    model.add(baseModel)
    model.add(AveragePooling2D(pool_size=(7, 7)))
    model.add(Flatten(name="flatten"))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(2, activation="sigmoid", name = 'Output'))

    
    for layer in baseModel.layers:
        layer.trainable = False
        
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=10, min_delta = .00075)
model_checkpoint = ModelCheckpoint('ModelWeights/Mobilenet1.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 5, mode = 'min')
epochs = 2000
batch_size = 32
    

dim = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
mobilenet = get_mobilenet(dim =dim)

    
augmentation =ImageDataGenerator(rotation_range = 20, width_shift_range = .2, height_shift_range = .2, 
                                                       horizontal_flip = True, shear_range = .15, 
                                 fill_mode = 'nearest', zoom_range = .15)
augmentation.fit(x_train)
mobilenet_history = mobilenet.fit_generator(augmentation.flow(x_train, y_train, batch_size = batch_size),
            epochs = epochs, 
     callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)